## Import Python Modules

In [1]:
#from google.auth import compute_engine
#credentials = compute_engine.Credentials(service_account_email = '751427818963-compute@developer.gserviceaccount.com', 
#                                         scopes = ['https://www.googleapis.com/auth/earthengine'])

import ee
#ee.Initialize(credentials, project='ccmeo-ag-000007')
#ee.Authenticate()
ee.Initialize()

import os
import sys

#Get the absolute path to the parent of current working directory 
cwd    = os.getcwd()
source_path = os.path.join(cwd, 'sources')
sys.path.append(source_path)
sys.path


import geemap
import Image
import ImgMask as IM
import LEAFNets as LFNs
import eoAuxData as eoAD
import eoTileGrids as eoTG
import eoParams


## Define a Input Directory

The execution of LEAF production tool requires a number of parameters. To facilitate the storage and passing of these parameters, Python Dictionary is utilized as the container. Specifically, there are SEVEN key:value pairs in the dictionary: 
   
   (1) 'sensor' : a integer sensor code representing a satellite sensor. The valid values for this code are 5, 7, 8, 9 or          101, which stand for Landsat 5,7,8,9 and Sentinel-2, respectively. 
   
   (2) 'year' : a 4 digits year integer, marking the year of image acquisition (e.g., 2020).  
   
   (3) 'months' : a list of month integers (e.g., [6,7,8]). With a list, Monthly biophysical products can be generated from one execution of the LEAF production tool. Obviously, any month integer in the list should in a valid range (1 to 12). Otherwise, the biophysical products corresponding to the peak season (June 15 to September 15) of a year (specified by 'year' key) will be produced.      
   
   (4) 'prod_names' : a list strings standing for different biophysical parameters. Currently, LEAF production tool can be used to generate a subset or a full set of FOUR biophysical parameters ['LAI', 'fCOVER', 'Albedo', 'fAPAR']. 
   
   (5) 'tile_names' : a list of strings representing different tiles. The basic spatial unit of the LEAF production tool is a tile, which is a 900km x 900km area and defined by Canadian tile griding system. Providing a list of tile name strings means that the biophysical parameter products for multiple tiles can be generated from one execution of this tool.  
   
   (6) 'spatial_scale' : an integer defining the spatial resolution (in meter) of product exporting.
   
   (7) 'location' : a string specifying the location of product exporting. There are only two valid values for this parameter, 'drive' and 'storage'. 
   
   (8) 'bucket' : the name string of a Google Cloud bucket. This parameter needs to be provided only when the value for 'location' key is defined by 'storage'.
   
   (9) 'out_folder' : the name string of the folder (on Google Drive or Google Cloud Storage) for exporting a set of biophysical parameter maps corresponding to one tile and one year. An empty string for this key means a folder name will be created automatically with a tile name and the acquisition year.   
   
Overall, there are NINE key:value pairs in the input parameter dictionary, six of them ('sensor', 'year', 'spatial_scale', 'location', 'bucket' and 'out_folder') expect a single value, while the other three need a list. With the different combinations of the three lists, various production scenarios can be carried out. For instance, to generate monthly (e.g., July) parameter maps for multiple tiles (e.g., 'tile41', 'tile42' and 'tile43'), two lists, [7] and ['tile41', 'tile42', 'tile43'], should be provided for 'months' and 'tile_names' keys.     

In [2]:
params = {
    'sensor': 'L8_SR',           # A sensor type code (e.g., 'S2_SR', 'L8_SR'5)
    'unit': 2,                   # A data unit code (1 or 2 for TOA or surface reflectance)
    'year': 2019,                # An integer representing image acquisition year
    'nbYears': -1,               # a positive or negative int for annual and monthly product, respectively
    'months': [8],              # A set of integers represening monthes (a negative value means whole summer)
    'tile_names': ['tile55'],    # A list of (sub-)tile names (defined using CCRS' tile griding system)     'prod_names': ['LAI','fAPAR','fCOVER','Albedo'],   #['LAI','fAPAR','fCOVER','Albedo', 'QC', 'date', 'partition']
    'prod_names': ['LAI'],    #['LAI','fAPAR','fCOVER','Albedo', 'QC', 'date', 'partition']
    'location': 'drive',         # Exporting location ('drive', 'storage' or 'asset') 
    'resolution': 30,            # Exporting spatial resolution
    'bucket': 's2_leaf_2021_date', # An unique bucket name on GCS that must have been created beforehand
    'folder': 'LS_leaf_product',  # the folder name for exporting
    'export_style': 'separate'
}


## Extract and Export Tiled Vegetation Parameter Maps

In [3]:
export_tasks = LFNs.LEAF_production(params)
#collName, prod_name, start, stop, mapBounds = LFCONS.LEAF_InputParams('s2b', 'LAI', '2020', '6', 'tile55')
#print(collName, prod_name, start, stop, mapBounds.getInfo())


<get_LC_map> Used LC map is:  projects/ccmeo-ag-000007/assets/CanLC2020


<getCollection> The name of data catalog =  LANDSAT/LC08/C02/T1_L2
<getCollection> The number of images in selected image collection =  87


<getCollection> The name of data catalog =  LANDSAT/LC08/C02/T1_TOA
<getCollection> The number of images in selected image collection =  89


<LS_separate_params> property names in used method =  ['biomeNumber', 'childFCList', 'RF', 'system:version', 'system:id', 'system:asset_size']
<LS_separate_params> unique biome values =  [1, 2, 3, 5, 6, 7]


## Check the Status of Exportings

In [4]:
Image.manage_tasks('status', '2021')


tile65_2021_Oct_Date_20m: PENDING
tile65_2021_Sep_Date_20m: PENDING
tile65_2021_Aug_Date_20m: PENDING
tile65_2021_Jul_Date_20m: PENDING
tile65_2021_Jun_Date_20m: PENDING
tile65_2021_May_Date_20m: PENDING
tile56_2021_Oct_Date_20m: PENDING
tile56_2021_Sep_Date_20m: PENDING
tile56_2021_Aug_Date_20m: PENDING
tile56_2021_Jul_Date_20m: PENDING
tile56_2021_Jun_Date_20m: PENDING
tile56_2021_May_Date_20m: PENDING
tile55_2021_Oct_Date_20m: PENDING
tile55_2021_Sep_Date_20m: PENDING
tile55_2021_Aug_Date_20m: PENDING
tile55_2021_Jul_Date_20m: PENDING
tile55_2021_Jun_Date_20m: PENDING
tile55_2021_May_Date_20m: PENDING
tile54_2021_Oct_Date_20m: PENDING
tile54_2021_Sep_Date_20m: PENDING
tile54_2021_Aug_Date_20m: PENDING
tile54_2021_Jul_Date_20m: PENDING
tile54_2021_Jun_Date_20m: PENDING
tile54_2021_May_Date_20m: PENDING
tile53_2021_Oct_Date_20m: PENDING
tile53_2021_Sep_Date_20m: PENDING
tile53_2021_Aug_Date_20m: PENDING
tile53_2021_Jul_Date_20m: PENDING
tile53_2021_Jun_Date_20m: PENDING
tile53_2021_Ma

## National Vegetation Parameter Extraction 

In [3]:
sensor = params['sensor']
unit   = params['unit']    
year   = params['year']        # A string to define a targeted year
nYears = params['nbYears']     # An integer to define the number of years

region = eoTG.custom_RegionDict.get(params['tile_names'][0])  #includes bigger full tiles

ssr_dict = Image.SSR_META_DICT[sensor]

print('ssr dictionary = ', ssr_dict)

ClassImg = eoAD.get_CanLC(2020)
can_mask = IM.Can_land_mask(2020, True).multiply(ee.Image(100))
bioMap = LFNs.National_LEAF_production(params)

ssr dictionary =  {'NAME': 'S2', 'SSR_CODE': 21, 'DATA_UNIT': 2, 'GAIN': <ee.ee_number.Number object at 0x0000027457A29C10>, 'OFFSET': <ee.ee_number.Number object at 0x0000027457A29C40>, 'ALL_BANDS': ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12'], 'OUT_BANDS': ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12'], 'SIX_BANDS': ['B2', 'B3', 'B4', 'B8A', 'B11', 'B12'], 'NoA_BANDS': ['B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12'], 'GEE_NAME': 'COPERNICUS/S2_SR_HARMONIZED', 'CLOUD': 'CLOUDY_PIXEL_PERCENTAGE', 'SZA': 'MEAN_SOLAR_ZENITH_ANGLE', 'VZA': 'MEAN_INCIDENCE_ZENITH_ANGLE_B8A', 'SAA': 'MEAN_SOLAR_AZIMUTH_ANGLE', 'VAA': 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A', 'BLU': 'B2', 'GRN': 'B3', 'RED': 'B4', 'NIR': 'B8A', 'SW1': 'B11', 'SW2': 'B12'}


<getCollection> The name of data catalog =  COPERNICUS/S2_SR_HARMONIZED
<getCollection> The number of images in selected image collection =  146029


<makeIndexLayer> land cover IDs =  [1, 3, 4, 17, 7, 8, 5, 1

In [4]:
Map = geemap.Map(center=(400, -100), zoom=4)
Map

Map(center=[400, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(…

In [6]:
param_vis = {
'min': 0,
'max': 100,
'palette': ['000000', '006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5', '555555', '999999', 'CCCCCC', 'FF0000', '00FF00', '0000FF']}

landsat_vis = {
    'min': 0,
    'max': 6000,
    'gamma': 1.6,
    'bands': ['B11', 'B8A', 'B4']
}

test_vis = {
    'min': 0,
    'max': 20,
    'gamma': 1.6,
    'bands': ['estimateLAI', 'partition', 'networkID']
}

test2_vis = {
    'min': -1,
    'max': 1,
    'gamma': 1.6,
    'bands': ['B11','B8A','B4']
}

#Map.addLayer(LEAF_cube, test_vis, 'LAI map')

Map.addLayer(can_mask, param_vis, 'Can mask')
Map.addLayer(ClassImg, param_vis, 'Can LC')
Map.addLayer(bioMap, param_vis, 'bio map')


AttributeError: 

The image argument in 'addLayer' function must be an instance of one of ee.Image, ee.Geometry, ee.Feature or ee.FeatureCollection.